In [1]:
%matplotlib inline

This is a five paremeter room RC model

dT = T(-dt/RC) + T_out(dt/RC) + E_lux(rho1*dt/C) + Occu(Q_int*dt/C) + Rads(Q_rad*dt/C) + q_vav(T_dat-T)*(rho2*dt/C) + dw

rho1: correlation between light intensity sensor and solar gain/electric lighting
rho2: correlation between the mass flow rate and volumetric flow rate
Simplified form:

dT = -T*x2 + T_out*x2 + E_lux*x3 + Occu * x4 + Rads *x5 - T*q_vav*x6 + T_sup * q_vav * x6 + x7


In [13]:
import numpy as np
import sympy
x1, x2, x3, x4, x5, x6, x7, u1, u2, u3, u4, u5, u6= sympy.symbols('x1 x2 x3 x4 x5 x6 x7 u1 u2 u3 u4 u5 u6')
X = sympy.Matrix([[x1],
                      [x2],
                      [x3],
                      [x4],
                      [x5],
                      [x6],
                      [x7]])
U = sympy.Matrix([[u1],
                      [u2],
                      [u3],
                      [u4],
                      [u5],
                      [u6]])
fun = sympy.Matrix([[x1-x1*x2+u1*x2+u2*x3+u3*x4+u4*x5+u5*u6*x6-x1*u6*x6+x7],
                          [x2],
                          [x3],
                          [x4],
                          [x5],
                          [x6],
                          [x7]])

In [14]:
P = np.eye(7)
q = np.array([0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])[np.newaxis]
Q = q.T * q
print(Q)

[[ 0.01  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]]


In [40]:
R = np.matrix([[0.3,0.02,0.01,0.01,0.01,0.01,0.01],
               [0.002,0.001,0.003,0.002,0.001,0.002,0.001],
               [0.02,0.04,0.04,0.02,0.03,0.01,0.02],
               [0.001,0.002,0.002,0.005,0.004,0.003,0.002],
              [0.0000,0.0002,0.0002,0.0005,0.0004,0.0003,0.0002],
              [0.0002,0.0002,0.0002,0.0005,0.0004,0.0003,0.0002],
              [0.0,0.0002,0.0002,0.0005,0.0005,0.0003,0.0],])
H = sympy.Matrix([x1, 0, 0, 0, 0, 0, 0])

In [37]:
import pandas as pd
import library.EKF as EKF
data = pd.read_csv('data/EP_Temperature.csv')
state_estimates = np.matrix([[data['IAT'][0]],
                            [0.0045],
                            [0.0003],
                            [0.01],
                            [0.1],
                            [0.0035],
                            [0.02]])

temp_ekf = EKF.ExtendedKalmanFilter(fun, X, U, P, Q, H, R, state_estimates) 

In [42]:
import math
predicted = []
actual = []
measured = []
estimates = []
length = data.shape[0]
iterations = length

for i in range(1, iterations):
    #generate control vector
    OAT = data['OAT'][i-1]
    Lux = data['Lux'][i-1]
    DAT = data["DAT"][i-1]
    SAP = math.sqrt(data['SAP'][i-1])
    RAD = data['RAD'][i-1]
    OCC = data["OCC"][i-1]
    controls = np.matrix([[OAT], 
                          [Lux], 
                          [OCC], 
                          [RAD],
                          [DAT],
                          [SAP]])

    #prediction
    temp_ekf.predict(controls)
    predicted.append(temp_ekf.X_values[0,0])
    estimates.append(temp_ekf.X_values.T.reshape(-1,).tolist())

    #update
    meas = data["IAT"][i]
    actual.append(data["IAT"][i])
    measured.append(meas)
    temp_ekf.update(np.matrix([[meas],[0],[0],[0],[0],[0],[0]]))

LinAlgError: Singular matrix